In [1]:
import keras
import tensorflow as tf

# Memory Pre-configuration
config = tf.compat.v1.ConfigProto(
    gpu_options=tf.compat.v1.GPUOptions(
        per_process_gpu_memory_fraction=0.9,
        allow_growth = True
    )
    # device_count = {'GPU': 1}
)
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

이번 포스트에서는 순환 신경망으로 시퀀스 데이터를 생성하는 방법을 살펴보겠습니다.

### 시퀀스 데이터를 어떻게 생성할까?

딥러닝에서 시퀀스 데이터를 생성하는 일반적인 방법은 이전 토큰을 입력으로 사용해서 시퀀스의 다음 1개 또는 몇개의 토큰을 예측하는 것입니다.
* "the cat is on ma"이란 입력이 주어지면 다음 글자인 타깃 "t"을 예측하도록 네트워크를 훈련합니다
* 텍스트 데이터를 다룰때 토큰은 보통 단어 또는 글자입니다
* 이전 토큰들이 주어졌을때 다음 토큰의 확률을 모델링할 수 있는 네트워크를 **언어 모델 (Language Model)**이라고 부릅니다

언어 모델을 훈련하고 나면 이 모델에서 샘플링 할 수 있습니다.
* 새로운 시퀀스를 생성합니다
* 초기 텍스트 문자열 (**조건 데이터 (conditioning data)**) 주입하고 새로운 글자나 단어를 생성합니다
* 생성된 출력은 다시 입력 데이터로 추가됩니다
* 이 과정을 여러번 반복합니다
* 이런 반복을 통해 모델이 훈련한 데이터 구조가 반영된 임의의 길이를 가진 시퀀스를 생성할 수 있습니다

이번 예제에서는 LSTM 층을 사용합니다
* 텍스트 Corpus에서 N개의 글자로 이루어진 문자열을 추출하여 주입하고 N+1 번째 글자를 예측하도록 훈련
* 모델의 출력은 출력 가능한 모든 글자에 해당하는 소프트맥스 값
* 즉 모델의 출력은 다음 글자의 확률 분포

이 LSTM을 **Character-level neural language model**이라고 부릅니다.

<br></br>
![](../images/8-1-languagemodel.png)
<br></br>

### 샘플링 전략의 중요성

텍스트를 생성할때 다음 글자를 선택하는 방법이 아주 중요합니다. 
* **Greedy Sampling**: 항상 가장 높은 확률을 가진 글자를 선택하는 방법
    * 반복적이고 예상 가능한 문자열을 만들기 때문에 논리적인 언어처럼 보이지 않음
* **Stochastic Sampling**: 다음 글자의 확률 분포에서 샘플링하는 과정에 무작위성을 주입하는 방법

모델의 소프트맥스 출력은 확률적 샘플링에 사용하기 좋습니다.
* 이따금 샘플링될 것같지 않은 글자를 샘플링
* 훈련 데이터에는 없지만 실제 같은 새로운 단어를 만들어 재미있고 창의적으로 보이는 문장을 생성
* 한가지 문제는 샘플랭 과정에서 무작위성의 양을 조절할 방법이 없음

샘플링 과정에서 확률의 양을 조절하기 위해 **소프트맥스 온도 (softmax temperature)**라는 파라미터를 사용합니다
* 이 파라미터는 샘플링에 사용되는 확률 분포의 엔트로피를 나타냄
* temperature 갑이 주어지면 다음과 같이 가중치를 적용하여 원본 확률 분포에서 새로운 확률 분포를 계산

##### 다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기

In [2]:
import numpy as np

## original_distribution은 전체 합이 1인 1D 넘파이 배열
def reweight_distribution(original_distribution, temperature = 0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    
    ## 원본 분포의 가중치를 변경하여 반환합니다
    return distribution / np.sum(distribution)

<br></br>
![](../images/8-1-temp.png)
<br></br>

낮은 온도는 무작위성이 낮기 때문에 예상할 수 있는 데이터를 생성합니다.

### 글자 수준의 LSTM 텍스트 생성 모델 구현

이번 예시에서는 19세기 후반 독일의 철학자 니체의 글을 사용하겠습니다.

#### 데이터 전처리

먼저 말뭉치를 내려받아 소문자로 바꿉니다.

##### 원본 데이터를 다운로드받아 파싱하기

In [3]:
import keras
import numpy as np

path = keras.utils.get_file("nietzsche.txt",
                            origin = "https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print("말뭉치 크기:", len(text))

말뭉치 크기: 600893


그 다음 maxlen 길이를 가진 시퀀스를 중복하여 추출합니다. 
* 추출한 시퀀스는 원-핫 인코딩으로 변환
* 크기가 (sequences, maxlen, unique_characters)인 3D 넘파이 배열 x로 합침
* 동시에 훈련 샘플에 상응하는 타깃을 담음 배열 y를 준비
* 타깃은 추출된 시퀀스 다음에 오는 원-핫 인코딩된 글자

##### 글자 시퀀스 벡터화하기

In [4]:
## 60개의 글자로 된 시퀀스를 추출
maxlen = 60

## 세 글자씩 건너뛰면서 새로운 시퀀스를 샘플링
step = 3

## 추출한 시퀀스를 담을 리스트
sentences = []

## 시퀀스 다음 글자를 담을 리스트
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print("시퀀스 개수:", len(sentences))

## Corpus에서 고유한 글자를 담은 리스트
chars = sorted(list(set(text)))

print("고유한 글자:", len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print("벡터화...")

x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 200278
고유한 글자: 57
벡터화...


#### 네트워크 구성

이 네트워크는 하나의 LSTM층과 그 뒤에 Dense 분류기가 뒤따릅니다.
* 분류기는 가능한 모든 글자에 대한 소프트맥스 출력을 만듬

##### 다음 글자를 예측하기 위한 단일 LSTM 모델

In [5]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation = "softmax"))

optimizer = keras.optimizers.RMSprop(lr = 0.01)
model.compile(loss = "categorical_crossentropy", optimizer = optimizer)

#### 언어 모델 훈련과 샘플링

훈련된 모델과 시드로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있습니다.

1. 지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다
2. 특정 온도로 이 확률 분포의 가중치를 조정합니다
3. 가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다
4. 새로운 글자를 생성된 텍스트의 끝에 추가합니다

다음 코드는 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출합니다

##### 모델의 예측이 주어졌을때 새로운 글자를 샘플링하는 함수

In [6]:
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

마지막으로 다음 반복문은 반복적으로 훈련하고 텍스트를 생성합니다.
* 에포크마다 학습이 끝난 후 여러가지 온도를 사용하여 텍스트를 생성합니다
* 이렇게 하면 모델이 수렴하면서 생성된 텍스트를 어떻게 잔화하는지 볼 수 있습니다
* 온도가 샘플링 전략에 미치는 영향을 보여줍니다

##### 텍스트 생성 루프

In [7]:
import random
import sys

random.seed(42)

start_index = random.randint(0, len(text) - maxlen - 1)

# for epoch in range(1, 60):
for epoch in range(1, 10):    
    
    print("에포크", epoch)
    
    ## 데이터에서 한번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size = 128, epochs = 1)
    
    ## 무작위로 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print("--- 시드 텍스트: ''" + seed_text + "'")
    
    ## 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print("------- 온도:", temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        ## 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
        for i in range(400):
            
            ## 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            preds = model.predict(sampled, verbose = 0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
1565/1565 [==============================] - 8s 4ms/step - loss: 2.2831
--- 시드 텍스트: ''the slowly ascending ranks and classes, in which,
through fo'
------- 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the says and the same the sense and seess to sense of the same to the sense and sense of the sense of the sense and the sense of the sanstering to the says and all the sense of the sense of the same the same the sense of the sense and to the same and all the sense and the sense and the same the sense and a sone the sense of the sense of the same and the sense and seeps and to the sense and and t
------- 온도: 0.5
the slowly ascending ranks and classes, in which,
through for ablouss, the self-casisince of the arsesseterition, a man selse of enduring as the propess be the seess and for the sense to the selses as the spare is the consequent to the sectual woold sense of the says of the save even of the self-resured of one more to have is self-as soul the sections

us jubtafer to will acquiring chuirsion who were hegorrmied, on our decente thetever, their minally suchurjence of

에포크 5
1565/1565 [==============================] - 5s 3ms/step - loss: 1.4459
--- 시드 텍스트: ''the slowly ascending ranks and classes, in which,
through fo'
------- 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the same the most superion to the spirit is always them and soul of the spirit is the states of the spirit is the spirit is all the souls of the same of the states of the spirit is all the states of the spirit of the spirit of the states of the spirit who is the experience, which is the sublime to the spirit is always the state of the state of many the state of the spirit of the one is not one i
------- 온도: 0.5
the slowly ascending ranks and classes, in which,
through fort of sublical
actions, which we who the depposite and sense that the sense of all believe because, it is lives more part of the great stronger of the spervent of provew of stat

i god was a reducth these most frament is weachid. it berupesh; perhaps recisivelus i also the motend with kypers or lits atthing instine for religion, was from
the same of since--their tenvess.


otwarn nationsy-por), it error, in peristors of the
seifed we victfwerming asoral, views without theses only there the

에포크 9
1565/1565 [==============================] - 6s 4ms/step - loss: 1.3776
--- 시드 텍스트: ''the slowly ascending ranks and classes, in which,
through fo'
------- 온도: 0.2
the slowly ascending ranks and classes, in which,
through former and superiority of the most desires of the spirit and soul the sense of the sense of the sense of the sense of the proper and the superstition of the most man should be one must be the sense of the self-proble will as a spirit also in the sense of the experience, and the fact of the sense of the sense of the sense of the most desires of the superstition of the sense of the spirit and strength
------- 온도: 0.5
the slowly ascending ranks and class

낮은 온도는 아주 반복적이고 예상되는 텍스트를 만듭니다. 하지만 국부적인 구조는 실제와 매우 같습니다. 특히 모든 단어가 실제 영어 단어입니다.

높은 온도에서 생성된 텍스트는 창의적이기도 합니다. "reprovial" 같은 이따금 그럴싸해보이는 완전히 새로운 단어를 창조합니다.